# Differential regulation: T-test for two groups

This API example shows the functionality in the [`acore.differential_regulation`](acore.differential_regulation) module:

- ANOVA (for two groups) is equivalent to t-test between two groups

Then we can do the same for examples with three
and more groups, where a omnibus analysis across groups
is combined with posthoc anlysis for separate groups.

The functions are the same for both cases. The `group1` and
`group2` columns give the posthoc comparison.

Using vuecore we add:
- [ ] include a PCA colored by groups as well as covariance factors

In [ ]:
%pip install acore

In [2]:
import dsp_pandas
import pandas as pd

import acore.differential_regulation as ad

dsp_pandas.format.set_pandas_options(
    max_columns=9,
    max_colwidth=20,
)

In [ ]:
BASE = (
    "https://raw.githubusercontent.com/Multiomics-Analytics-Group/acore/"
    "updt_diff_reg_api_example/example_data/alzheimer_proteomics/"
)
# data is already preprocessed: log2, filtered
fname: str = "alzheimer_example_omics_and_clinic.csv"  # combined omics and meta data
covariates: list[str] = ["age", "male"]
group: str = "AD"
subject_col: str = "Sample ID"
drop_cols: list[str] = []
factor_and_covars: list[str] = [group, *covariates]

Alternatively you can use the metabolomics dataset MTBLS13311,
which is also preprocessed and combined.

In [ ]:
# BASE = (
#     "https://raw.githubusercontent.com/Multiomics-Analytics-Group/acore/"
#     "updt_diff_reg_api_example/example_data/MTBLS13311/"
# )
# fname: str = "MTBLS13411_omics_and_meta.csv"  # combined omics and meta data
# covariates: list[str] = []
# group: str = "Factor Value[Strain type]"
# drop_cols: list[str] = ["group"]
# subject_col: str | int = 0
# factor_and_covars: list[str] = [group, *covariates]

# ANCOVA analysis for two groups
Use combined dataset for ANCOVA analysis.

In [ ]:
omics_and_clinic = (
    pd.read_csv(f"{BASE}/{fname}", index_col=subject_col)
    .convert_dtypes()
    .dropna(subset=factor_and_covars)
)
omics_and_clinic

Drop unnecessary columns, if there are any specified in `drop_cols`.

In [ ]:
if drop_cols:
    omics_and_clinic.drop(columns=drop_cols, inplace=True)
omics_and_clinic

metadata here is of type integer. All floats are proteomics measurements.

In [ ]:
omics_and_clinic.dtypes.value_counts()

In [ ]:
omics_and_clinic[[group, *covariates]]

# ANOVA analysis for two groups
not controlling for covariates
> To check: pvalues for proteins with missing mean values? some merging issue?

In [ ]:
if not isinstance(subject_col, str):
    subject_col = omics_and_clinic.index.name or "index"
    omics_and_clinic.rename_axis(subject_col, axis=0, inplace=True)
anova = (
    ad.run_anova(
        omics_and_clinic.reset_index(),
        subject=subject_col,
        drop_cols=covariates,
        group=group,
    )
    .set_index("identifier")
    .sort_values(by="padj")
)
anova

In [ ]:
# ToDo: Something is wrong with the mean and std dev calculations for each group
# FC is also calculated along means and std dev.
anova.describe().T

Set subject to None

In [ ]:
anova = (
    ad.run_anova(
        omics_and_clinic,
        subject=None,
        drop_cols=covariates,
        group=group,
    )
    .set_index("identifier")
    .sort_values(by="padj")
)
anova

view averages per protein group

In [ ]:
view = anova.iloc[:, 2:7]
viewed_cols = view.columns.to_list()
view

Test results

In [ ]:
regex_filter = "pval|padj|reject|stat|FC"
view = anova.filter(regex=regex_filter)
viewed_cols.extend(view.columns)
view

Other information

In [ ]:
anova.drop(columns=viewed_cols)

Done.